In [1]:
import pandas as pd
import psycopg2
import numpy as np

In [2]:
raw = pd.read_csv(r'C:\Users\erico\Documents\projeto-regressao\Supermarket-store-sales-prediction\data\raw\Stores.csv', delimiter=';')

In [3]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Store ID              896 non-null    int64
 1   Store_Area            896 non-null    int64
 2   Items_Available       896 non-null    int64
 3   Daily_Customer_Count  896 non-null    int64
 4   Store_Sales           896 non-null    int64
dtypes: int64(5)
memory usage: 35.1 KB


In [5]:
# Conectar ao banco de dados
conn = psycopg2.connect(
    host="localhost",
    database="Supermarket",
    user="postgres",
    password="123456"
)

# Abrir um cursor para executar consultas
cur = conn.cursor()
cur.execute("ROLLBACK")

# Executar uma consulta SQL para verificar se a tabela já existe
cur.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM   information_schema.tables 
        WHERE  table_name = 'Supermarket_Gold'
    );
""")

# Recuperar o resultado da consulta
table_exists = cur.fetchone()[0]

# Se a tabela não existir, criar a tabela
if not table_exists:
    cur.execute("""
        CREATE TABLE Supermarket_Gold (
            store_id INT PRIMARY KEY,
            store_area INT,
            items_available INT,
            daily_customer_count INT,
            store_sales INT
        );
    """)
    conn.commit()

DuplicateTable: relation "supermarket_gold" already exists


In [7]:
# Popular banco de dados

# Loop através das linhas do dataframe e inserir cada linha na tabela
for index, row in raw.iterrows():
    cur.execute("""
        INSERT INTO Supermarket_Gold (store_id, store_area, items_available, daily_customer_count, store_sales)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (
        int(row['Store ID ']),
        int(row['Store_Area']),
        int(row['Items_Available']),
        int(row['Daily_Customer_Count']),
        int(row['Store_Sales'])
    ))

# Salvar as alterações no banco de dados
conn.commit()

# Fechar o cursor e a conexão com o banco de dados
cur.close()
conn.close()